## Random Forest

## Install and import necessary packages

In [22]:
# You may need to install xgboost (it's not part of the sklearn package)
# !conda install xgboost 

In [23]:
# import packages
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from xgboost import XGBClassifier

np.random.seed(1)

## Load data 

In [24]:
X_train = pd.read_csv('data/ub_train_X.csv')
y_train = pd.read_csv('data/ub_train_y.csv')
X_test = pd.read_csv('data/ub_test_X.csv')
y_test = pd.read_csv('data/ub_test_y.csv')

In [25]:
performance = pd.DataFrame({"model": [], "Accuracy": [], "Precision": [], "Recall": [], "F1": []})

## Prediction with RandomForest (using default parameters)

* n_estimators: The number of trees in the forsest
    - A deeper tree might increase the performance, but also the complexity and chances to overfit.
    - The value must be an integer greater than 0. Default is 100.  
* max_depth: The maximum depth per tree. 
    - Deeper trees might increase the performance, but also the complexity and chances to overfit.
    - The value must be an integer greater than 0. Default is None, which allows the tree to grow without constraint.

In [26]:
rforest = RandomForestClassifier()

In [27]:
_ = rforest.fit(X_train, y_train)

C:\Users\mukes\AppData\Local\Temp\ipykernel_35876\139911232.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  _ = rforest.fit(X_train, y_train)


In [28]:
y_pred = rforest.predict(X_test)

In [29]:
model_preds = rforest.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Random_forest", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,Random_forest,0.983333,0.984375,0.845638,0.909747


In [30]:
rforest.get_params

<bound method BaseEstimator.get_params of RandomForestClassifier()>

## Random forest with random search

In [31]:
score_measure = "recall"
rf = RandomForestClassifier()
kfolds = 5
param_grid = {
    'min_samples_split': [2,3,4,5,6,8,10],  
    'min_samples_leaf': [1, 2,3, 4],
    'max_leaf_nodes': np.arange(5, 250), 
    'max_features' : ['auto','sqrt'],
    'bootstrap' : [True , False],
    'max_depth': [10,20,30,40,50,60,70,None]
} 
  
grid = RandomizedSearchCV(rf, param_grid, refit = True, verbose = 3)
  
# fitting the model for grid search
grid.fit(X_train, y_train)
print(f"The best {score_measure} score is {grid.best_score_}")
print(f"... with parameters: {grid.best_params_}")

bestRecallTree = grid.best_estimator_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


c:\Users\mukes\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV 1/5] END bootstrap=True, max_depth=60, max_features=sqrt, max_leaf_nodes=195, min_samples_leaf=4, min_samples_split=6;, score=0.989 total time=   0.2s


c:\Users\mukes\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV 2/5] END bootstrap=True, max_depth=60, max_features=sqrt, max_leaf_nodes=195, min_samples_leaf=4, min_samples_split=6;, score=0.984 total time=   0.2s


...

[CV 2/5] END bootstrap=True, max_depth=70, max_features=sqrt, max_leaf_nodes=30, min_samples_leaf=1, min_samples_split=3;, score=0.979 total time=   0.1s
[CV 3/5] END bootstrap=True, max_depth=70, max_features=sqrt, max_leaf_nodes=30, min_samples_leaf=1, min_samples_split=3;, score=0.980 total time=   0.1s


c:\Users\mukes\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\Users\mukes\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV 4/5] END bootstrap=True, max_depth=70, max_features=sqrt, max_leaf_nodes=30, min_samples_leaf=1, min_samples_split=3;, score=0.990 total time=   0.1s
[CV 5/5] END bootstrap=True, max_depth=70, max_features=sqrt, max_leaf_nodes=30, min_samples_leaf=1, min_samples_split=3;, score=0.989 total time=   0.1s


c:\Users\mukes\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\Users\mukes\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:926: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


The best recall score is 0.9888571428571428
... with parameters: {'min_samples_split': 2, 'min_samples_leaf': 3, 'max_leaf_nodes': 161, 'max_features': 'sqrt', 'max_depth': 50, 'bootstrap': False}


In [32]:
grid.get_params

<bound method BaseEstimator.get_params of RandomizedSearchCV(estimator=RandomForestClassifier(),
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'max_leaf_nodes': array([  5,   6,   7,   8,   9,  10,  11,  12,  13,  14,  15,  16,  17,
        18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,
        31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,
        44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,
        57,  5...
       174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186,
       187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199,
       200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212,
       213, 214, 215, 216, 217, 218, 2

In [33]:
model_preds = bestRecallTree.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Random_Rforest", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,Random_forest,0.983333,0.984375,0.845638,0.909747
0,Random_Rforest,0.981333,0.968992,0.838926,0.899281


### Features

In [34]:
np.round(grid.best_estimator_.feature_importances_,2)

array([0.03, 0.02, 0.33, 0.13, 0.18, 0.03, 0.  , 0.06, 0.  , 0.01, 0.11,
       0.1 ])

## RF with Grid

In [35]:
score_measure = "recall"
rf = RandomForestClassifier()
kfolds = 5
param_grid = {
    'min_samples_split': [2,3],  
    'min_samples_leaf': [1, 2],
    'max_leaf_nodes': np.arange(5, 8), 
    'max_features' : ['auto','sqrt'],
    'bootstrap' : [True , False],
    'max_depth': [10,None]
} 
  
grid = GridSearchCV(rf, param_grid, refit = True, verbose = 3)
  
# fitting the model for grid search
grid.fit(X_train, y_train)
print(f"The best {score_measure} score is {grid.best_score_}")
print(f"... with parameters: {grid.best_params_}")

bestRecallTree = grid.best_estimator_

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV 1/5] END bootstrap=True, max_depth=10, max_features=auto, max_leaf_nodes=5, min_samples_leaf=1, min_samples_split=2;, score=0.939 total time=   0.1s


c:\Users\mukes\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\Users\mukes\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV 2/5] END bootstrap=True, max_depth=10, max_features=auto, max_leaf_nodes=5, min_samples_leaf=1, min_samples_split=2;, score=0.940 total time=   0.0s
[CV 3/5] END bootstrap=True, max_depth=10, max_features=auto, max_leaf_nodes=5, min_samples_leaf=1, min_samples_split=2;, score=0.941 total time=   0.0s


c:\Users\mukes\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\Users\mukes\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV 4/5] END bootstrap=True, max_depth=10, max_features=auto, max_leaf_nodes=5, min_samples_leaf=1, min_samples_split=2;, score=0.937 total time=   0.1s
[CV 5/5] END bootstrap=True, max_depth=10, max_features=auto, max_leaf_nodes=5, min_samples_leaf=1, min_samples_split=2;, score=0.931 total time=   0.0s


...

[CV 2/5] END bootstrap=False, max_depth=None, max_features=sqrt, max_leaf_nodes=7, min_samples_leaf=2, min_samples_split=3;, score=0.954 total time=   0.0s
[CV 3/5] END bootstrap=False, max_depth=None, max_features=sqrt, max_leaf_nodes=7, min_samples_leaf=2, min_samples_split=3;, score=0.941 total time=   0.0s


c:\Users\mukes\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\Users\mukes\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV 4/5] END bootstrap=False, max_depth=None, max_features=sqrt, max_leaf_nodes=7, min_samples_leaf=2, min_samples_split=3;, score=0.949 total time=   0.0s
[CV 5/5] END bootstrap=False, max_depth=None, max_features=sqrt, max_leaf_nodes=7, min_samples_leaf=2, min_samples_split=3;, score=0.947 total time=   0.0s


c:\Users\mukes\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\Users\mukes\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:926: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


The best recall score is 0.9520000000000002
... with parameters: {'bootstrap': True, 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': 7, 'min_samples_leaf': 1, 'min_samples_split': 3}


In [36]:
model_preds = bestRecallTree.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Grid_Rforest", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,Random_forest,0.983333,0.984375,0.845638,0.909747
0,Random_Rforest,0.981333,0.968992,0.838926,0.899281
0,Grid_Rforest,0.933333,1.000000,0.328859,0.494949


### Features

In [37]:
np.round(grid.best_estimator_.feature_importances_,2)

array([0.  , 0.01, 0.33, 0.12, 0.21, 0.04, 0.  , 0.12, 0.  , 0.  , 0.08,
       0.08])

In [38]:
grid.get_params

<bound method BaseEstimator.get_params of GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'bootstrap': [True, False], 'max_depth': [10, None],
                         'max_features': ['auto', 'sqrt'],
                         'max_leaf_nodes': array([5, 6, 7]),
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 3]},
             verbose=3)>

## Prediction with ADABoost (using default parameters)

In [39]:
aboost = AdaBoostClassifier()
_ = aboost.fit(X_train, y_train)
y_pred = aboost.predict(X_test)

c:\Users\mukes\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [40]:
model_preds = aboost.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Adaboost_Rforest", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance


,model,Accuracy,Precision,Recall,F1
0,Random_forest,0.983333,0.984375,0.845638,0.909747
0,Random_Rforest,0.981333,0.968992,0.838926,0.899281
0,Grid_Rforest,0.933333,1.000000,0.328859,0.494949
0,Adaboost_Rforest,0.962667,0.878049,0.724832,0.794118


In [41]:
np.round(grid.best_estimator_.feature_importances_,2)

array([0.  , 0.01, 0.33, 0.12, 0.21, 0.04, 0.  , 0.12, 0.  , 0.  , 0.08,
       0.08])

In [42]:
grid.get_params

<bound method BaseEstimator.get_params of GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'bootstrap': [True, False], 'max_depth': [10, None],
                         'max_features': ['auto', 'sqrt'],
                         'max_leaf_nodes': array([5, 6, 7]),
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 3]},
             verbose=3)>

## XGBoost

In [43]:
xgboost = XGBClassifier()
_ = xgboost.fit(X_train, y_train)
y_pred = xgboost.predict(X_test)

In [44]:
model_preds = xgboost.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"XG_boost_Rforest", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance


,model,Accuracy,Precision,Recall,F1
0,Random_forest,0.983333,0.984375,0.845638,0.909747
0,Random_Rforest,0.981333,0.968992,0.838926,0.899281
0,Grid_Rforest,0.933333,1.000000,0.328859,0.494949
0,Adaboost_Rforest,0.962667,0.878049,0.724832,0.794118
0,XG_boost_Rforest,0.985333,0.956835,0.892617,0.923611


In [45]:
np.round(grid.best_estimator_.feature_importances_,2)

array([0.  , 0.01, 0.33, 0.12, 0.21, 0.04, 0.  , 0.12, 0.  , 0.  , 0.08,
       0.08])

In [46]:
grid.get_params

<bound method BaseEstimator.get_params of GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'bootstrap': [True, False], 'max_depth': [10, None],
                         'max_features': ['auto', 'sqrt'],
                         'max_leaf_nodes': array([5, 6, 7]),
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 3]},
             verbose=3)>

In [47]:
performance.sort_values(by='Accuracy')

,model,Accuracy,Precision,Recall,F1
0,Grid_Rforest,0.933333,1.000000,0.328859,0.494949
0,Adaboost_Rforest,0.962667,0.878049,0.724832,0.794118
0,Random_Rforest,0.981333,0.968992,0.838926,0.899281
0,Random_forest,0.983333,0.984375,0.845638,0.909747
0,XG_boost_Rforest,0.985333,0.956835,0.892617,0.923611
